In [1]:
import pandas as pd
import numpy as np
import unicodedata
import difflib
import unidecode

In [2]:
platforms = ["Guardian", "NME1", "Pitchfork", "Spectrum", "DIY", "LAQ", "Gigwise", "Ohm", "Skinny", "Uncut", "UTR"]
paths = [f"C:\\Users\\tommy\\OneDrive\\University\\Year 3\\Third Year Project\\Platform Album Data\\{platform}.csv" for platform in platforms]

In [3]:
dfs = {platform: pd.read_csv(path, index_col=0) for platform, path in zip(platforms, paths)}

In [4]:
{platform: df.shape[0] for platform, df in dfs.items()}

{'Guardian': 183479,
 'NME1': 5052,
 'Pitchfork': 25396,
 'Spectrum': 7933,
 'DIY': 2234,
 'LAQ': 2153,
 'Gigwise': 1922,
 'Ohm': 1488,
 'Skinny': 3734,
 'Uncut': 10969,
 'UTR': 4200}

In [5]:
dfs = {platform: df.drop_duplicates() for platform, df in dfs.items()}

for platform, df in dfs.items():
    
    # df.drop(columns=["Platform"], axis=1, inplace=True)
    df.loc[:, "Album"] = df["Album"].apply(lambda x: unidecode.unidecode(unicodedata.normalize("NFKD", str(x))))
    df.loc[:, "Artist"] = df["Artist"].apply(lambda x: unidecode.unidecode(unicodedata.normalize("NFKD", str(x))))
    df.loc[:, "Album"] = df["Album"].apply(lambda x: x.strip("'`").replace(".", "").replace(",", "").replace("–", "").replace("-", "").replace(
        "&", "And").lower().replace("' review", "").strip())
    df.loc[:, "Artist"] = df["Artist"].apply(lambda x: x.strip("'`").replace(".", "").replace(",", "").replace("–", "").replace("-", "").replace(
        "&", "And").replace("album review:", "").lower().strip())
    df.rename({"Url": f"{platform}_url"}, axis=1, inplace=True)
    dfs[platform] = df


In [35]:
platforms = list(dfs.keys())
p1 = []
p2 = []
intersect = []

for i in range(len(dfs)):
    for j in range(len(dfs)):
        platform1 = platforms[i]
        platform2 = platforms[j]
        n = pd.merge(dfs[platform1], dfs[platform2], how="inner", on=["Album", "Artist"]).shape[0]
        p1.append(platform1)
        p2.append(platform2)
        intersect.append(n)

data = pd.DataFrame({"Platform1": p1, "Platform2": p2, "n": intersect})
data.loc[data["Platform1"] == data["Platform2"]] = None
data.loc[data["Platform1"] == "NME1", "Platform1"] = "NME"
data.loc[data["Platform2"] == "NME1", "Platform2"] = "NME"
        
pd.crosstab(data.loc[:, "Platform1"], data.loc[:, "Platform2"], intersect, aggfunc=sum).style.background_gradient(axis=None).format(precision=0)

Platform2,DIY,Gigwise,Guardian,LAQ,NME,Ohm,Pitchfork,Skinny,Spectrum,UTR,Uncut
Platform1,,,,,,,,,,,
DIY,nan,407,1091,729,1043,488,1285,707,900,693,133
Gigwise,407,nan,316,269,433,295,375,155,279,201,50
Guardian,1091,316,nan,715,1769,571,2931,1128,1954,1027,709
LAQ,729,269,715,nan,536,510,1070,530,654,477,140
NME,1043,433,1769,536,nan,520,2056,879,1339,826,270
Ohm,488,295,571,510,520,nan,717,291,547,338,121
Pitchfork,1285,375,2931,1070,2056,717,nan,1605,4035,1782,1167
Skinny,707,155,1128,530,879,291,1605,nan,1114,707,211
Spectrum,900,279,1954,654,1339,547,4035,1114,nan,1254,510


In [43]:
sample = ["Spectrum", "NME1", "Pitchfork"]
sample_df = dfs["Guardian"]

for s in sample:
    sample_df = pd.merge(sample_df, dfs[s], on=["Artist", "Album"], how="inner")

sample_df.shape[0]

C:\Users\tommy\AppData\Local\Temp\ipykernel_18276\2556974383.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Platform_x'} in the result is deprecated and will raise a MergeError in a future version.
  sample_df = pd.merge(sample_df, dfs[s], on=["Artist", "Album"], how="inner")


801

In [48]:
sample1 = ["Spectrum", "NME1", "Pitchfork", "UTR"]
sample_df1 = dfs["Guardian"]

for s in sample1:
    sample_df1 = pd.merge(sample_df1, dfs[s], on=["Artist", "Album"], how="inner")
    
sample_df1.shape[0]

C:\Users\tommy\AppData\Local\Temp\ipykernel_18276\2548377270.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Platform_x'} in the result is deprecated and will raise a MergeError in a future version.
  sample_df1 = pd.merge(sample_df1, dfs[s], on=["Artist", "Album"], how="inner")


378

In [49]:
sample2 = ["Spectrum", "NME1", "Pitchfork", "Skinny"]
sample_df2 = dfs["Guardian"]

for s in sample2:
    sample_df2 = pd.merge(sample_df2, dfs[s], on=["Artist", "Album"], how="inner")
    
sample_df2.shape[0]

C:\Users\tommy\AppData\Local\Temp\ipykernel_18276\2416784739.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Platform_x'} in the result is deprecated and will raise a MergeError in a future version.
  sample_df2 = pd.merge(sample_df2, dfs[s], on=["Artist", "Album"], how="inner")


359

In [50]:
print(f"UTR: {1 - (378 / 801)}, Skinny: {1 - (359 / 801)}")

UTR: 0.5280898876404494, Skinny: 0.5518102372034956


In [24]:
df = dfs["Pitchfork"]

for key in list(dfs.keys())[1:]:
    
    pf_merged = pd.merge(df, dfs[key], how="inner", on=["Album", "Artist"])
    print(key, df.shape[0])
    print(pf_merged)


NME1 24549
     Platform_x          Artist                              Album Platform_y
0     Pitchfork        the 1975  being funny in a foreign language        NME
1     Pitchfork            tsha                      capricorn sun        NME
2     Pitchfork          daphni                             cherry        NME
3     Pitchfork         shygirl                              nymph        NME
4     Pitchfork          pixies                           doggerel        NME
...         ...             ...                                ...        ...
2051  Pitchfork      kanye west                  late registration        NME
2052  Pitchfork   kings of leon               aha shake heartbreak        NME
2053  Pitchfork  the libertines                     the libertines        NME
2054  Pitchfork            blur                         think tank        NME
2055  Pitchfork     the streets           original pirate material        NME

[2056 rows x 4 columns]
Pitchfork 24549
      Platfo